The first step is configuring the notebook for access to the IBM Cloud OpenStack Swift based ObjectStore since we want to store the final result in a JSON file to make it accessible from other notebooks

In [1]:
import org.apache.spark.sql.SparkSession

// @hidden_cell
// This function is used to setup the access of Spark to your Object Storage. The definition contains your credentials.
// You might want to remove those credentials before you share your notebook.
def setHadoopConfigd3bd5b94a9334de59a55a7fed2bedeaa(name: String) = {
    // This function sets the Hadoop configuration so it is possible to
    // access data from Bluemix Object Storage using Spark

    val prefix = "fs.swift.service." + name
    sc.hadoopConfiguration.set(prefix + ".auth.url", "https://identity.open.softlayer.com" + "/v3/auth/tokens")
    sc.hadoopConfiguration.set(prefix + ".auth.endpoint.prefix","endpoints")
    sc.hadoopConfiguration.set(prefix + ".tenant", "6aaf54352357483486ee2d4981f8ef15")
    sc.hadoopConfiguration.set(prefix + ".username", "c0eebedc019f4413be3f3d656821b35f")
    sc.hadoopConfiguration.set(prefix + ".password", "ji[T[l.(7D&gld*5")
    sc.hadoopConfiguration.setInt(prefix + ".http.port", 8080)
    sc.hadoopConfiguration.set(prefix + ".region", "dallas")
    sc.hadoopConfiguration.setBoolean(prefix + ".public", false)
}

// you can choose any name
val name = "keystone"
setHadoopConfigd3bd5b94a9334de59a55a7fed2bedeaa(name)

val spark = SparkSession.
    builder().
    getOrCreate()

We download the zip file containing the data to the local stagin area

In [2]:
import sys.process._
"wget http://www.femto-st.fr/f/d/Training_set.zip" !

--2017-06-27 17:31:57--  http://www.femto-st.fr/f/d/Training_set.zip
Resolving www.femto-st.fr (www.femto-st.fr)... 195.83.19.10
Connecting to www.femto-st.fr (www.femto-st.fr)|195.83.19.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 140424205 (134M) [application/zip]
Saving to: ‘Training_set.zip.1’

     0K .......... .......... .......... .......... ..........  0% 96.4K 23m42s
    50K .......... .......... .......... .......... ..........  0%  384K 14m49s
   100K .......... .......... .......... .......... ..........  0%  385K 11m51s
   150K .......... .......... .......... .......... ..........  0%  385K 10m22s
   200K .......... .......... .......... .......... ..........  0% 81.9M 8m18s
   250K .......... .......... .......... .......... ..........  0% 7.70M 6m58s
   300K .......... .......... .......... .......... ..........  0%  405K 6m46s
   350K .......... .......... .......... .......... ..........  0% 7.23M 5m57s
   400K .......... .......... ...

In [3]:
import sys.process._
"df -h" !

Filesystem      Size  Used Avail Use% Mounted on
/dev/sda3       930G  103G  827G  12% /
devtmpfs        189G     0  189G   0% /dev
tmpfs           189G  1.3M  189G   1% /dev/shm
tmpfs           189G  4.2G  185G   3% /run
tmpfs           189G     0  189G   0% /sys/fs/cgroup
/dev/sdd1       3.7T   33M  3.7T   1% /disk3
/dev/sdf1       3.7T   33M  3.7T   1% /disk5
/dev/sdi1       3.7T   33M  3.7T   1% /disk8
/dev/sdh1       3.7T   33M  3.7T   1% /disk7
/dev/sdc1       3.7T   33M  3.7T   1% /disk2
/dev/sde1       3.7T   33M  3.7T   1% /disk4
/dev/sdg1       3.7T   33M  3.7T   1% /disk6
/dev/sdb1       3.7T   11G  3.7T   1% /disk1
/dev/sdj1       3.7T   33M  3.7T   1% /disk9
/dev/sda1       253M  163M   91M  65% /boot
tmpfs            38G     0   38G   0% /run/user/0
/dev/fs01       246T  180T   67T  74% /gpfs/global_fs01


In [6]:
"unzip  ./Training_set.zip" !

Archive:  ./Training_set.zip
   creating: Learning_set/
   creating: Learning_set/Bearing1_1/
  inflating: Learning_set/Bearing1_1/acc_00001.csv  
  inflating: Learning_set/Bearing1_1/acc_00002.csv  
  inflating: Learning_set/Bearing1_1/acc_00003.csv  
  inflating: Learning_set/Bearing1_1/acc_00004.csv  
  inflating: Learning_set/Bearing1_1/acc_00005.csv  
  inflating: Learning_set/Bearing1_1/acc_00006.csv  
  inflating: Learning_set/Bearing1_1/acc_00007.csv  
  inflating: Learning_set/Bearing1_1/acc_00008.csv  
  inflating: Learning_set/Bearing1_1/acc_00009.csv  
  inflating: Learning_set/Bearing1_1/acc_00010.csv  
  inflating: Learning_set/Bearing1_1/acc_00011.csv  
  inflating: Learning_set/Bearing1_1/acc_00012.csv  
  inflating: Learning_set/Bearing1_1/acc_00013.csv  
  inflating: Learning_set/Bearing1_1/acc_00014.csv  
  inflating: Learning_set/Bearing1_1/acc_00015.csv  
  inflating: Learning_set/Bearing1_1/acc_00016.csv  
  inflating: Learning_set/Bearing1_1/acc_00017.csv  
  inf

In [5]:
"rm -Rf ./Learning_set" !

In [4]:
val bearing1_1_acc = spark.read.option("inferSchema","true").csv("./Learning_set/Bearing1_1/acc*")

In [5]:
bearing1_1_acc.printSchema

root
 |-- _c0: integer (nullable = true)
 |-- _c1: integer (nullable = true)
 |-- _c2: integer (nullable = true)
 |-- _c3: decimal(5,-1) (nullable = true)
 |-- _c4: double (nullable = true)
 |-- _c5: double (nullable = true)



In [6]:
bearing1_1_acc.show

+---+---+---+---------+------+------+
|_c0|_c1|_c2|      _c3|   _c4|   _c5|
+---+---+---+---------+------+------+
|  9| 38| 46|8.6566E+5|-1.626|-0.086|
|  9| 38| 46|8.6570E+5|-1.538|-0.299|
|  9| 38| 46|8.6574E+5|-0.969|-0.025|
|  9| 38| 46|8.6578E+5|-0.577| 0.008|
|  9| 38| 46|8.6582E+5| 0.143|-0.087|
|  9| 38| 46|8.6586E+5| 0.129|-0.611|
|  9| 38| 46|8.6590E+5| 0.636|-0.496|
|  9| 38| 46|8.6594E+5|-0.129| 0.588|
|  9| 38| 46|8.6598E+5|-0.323| 0.369|
|  9| 38| 46|8.6602E+5|-0.812| 0.019|
|  9| 38| 46|8.6605E+5|  -0.8| 0.642|
|  9| 38| 46|8.6609E+5|-0.845|-0.047|
|  9| 38| 46|8.6613E+5|-0.723| 0.117|
|  9| 38| 46|8.6617E+5|-0.527| 0.237|
|  9| 38| 46|8.6621E+5|-0.224| 0.334|
|  9| 38| 46|8.6625E+5|  0.23|-0.472|
|  9| 38| 46|8.6629E+5|-0.121|-0.551|
|  9| 38| 46|8.6633E+5|-0.258| -0.11|
|  9| 38| 46|8.6637E+5|-0.892| 0.039|
|  9| 38| 46|8.6641E+5|-0.948|-0.067|
+---+---+---+---------+------+------+
only showing top 20 rows



In [7]:
val hadoopConf = new org.apache.hadoop.conf.Configuration()
val hdfs = org.apache.hadoop.fs.FileSystem.get(new java.net.URI("swift://coursera." + name), hadoopConf)
hdfs.delete(new org.apache.hadoop.fs.Path("swift://coursera." + name + "/bearing1_1_acc.json"), true) 

Name: com.ibm.stocator.fs.common.exception.ConfigurationParseException
Message: Configuration parse exception: Missing mandatory configuration: .auth.url
StackTrace:   at com.ibm.stocator.fs.common.Utils.updateProperty(Utils.java:183)
  at com.ibm.stocator.fs.swift.ConfigurationHandler.initialize(ConfigurationHandler.java:80)
  at com.ibm.stocator.fs.swift.SwiftAPIClient.initiate(SwiftAPIClient.java:191)
  at com.ibm.stocator.fs.ObjectStoreVisitor.getStoreClient(ObjectStoreVisitor.java:124)
  at com.ibm.stocator.fs.ObjectStoreFileSystem.initialize(ObjectStoreFileSystem.java:95)
  at org.apache.hadoop.fs.FileSystem.createFileSystem(FileSystem.java:2669)
  at org.apache.hadoop.fs.FileSystem.access$200(FileSystem.java:94)
  at org.apache.hadoop.fs.FileSystem$Cache.getInternal(FileSystem.java:2703)
  at org.apache.hadoop.fs.FileSystem$Cache.get(FileSystem.java:2685)
  at org.apache.hadoop.fs.FileSystem.get(FileSystem.java:373)

In [ ]:
bearing1_1_acc.printSchema

In [10]:
bearing1_1_acc.createOrReplaceTempView("bearing1_1_acc")
val bearing1_1_acc_transformed = spark.sql("""
    SELECT concat(_c0,_c1,_c2) as cluster, 
    (cast(timestamp(concat('1970-01-01 ',_c0,':',_c1,':',_c2,'.123')) as long) *1000000)+_c3 as ts,
    _c4 as hacc,
    _c5 as vacc
    from bearing1_1_acc
""")

In [13]:
bearing1_1_acc_transformed.select("cluster").distinct().show

+-------+
|cluster|
+-------+
|  95519|
| 102539|
| 132159|
|  12529|
|   1159|
| 131949|
| 151859|
| 121849|
|  15269|
| 111919|
| 142039|
|  14369|
| 161929|
|  16639|
|  10309|
|  15539|
| 101149|
| 103819|
| 111349|
| 114959|
+-------+
only showing top 20 rows



In [13]:
bearing1_1_acc_transformed.write.json("swift://coursera." + name + "/bearing1_1_acc_transformed4.json")

In [ ]:
val bearing1_1_temp = spark.read.option("inferSchema","true").csv("./Learning_set/Bearing1_1/temp*")

In [ ]:
val hadoopConf = new org.apache.hadoop.conf.Configuration()
val hdfs = org.apache.hadoop.fs.FileSystem.get(new java.net.URI("swift://coursera." + name), hadoopConf)
hdfs.delete(new org.apache.hadoop.fs.Path("swift://coursera." + name + "/bearing1_1_temp.json"), true) 

In [ ]:
bearing1_1_temp.write.json("swift://coursera." + name + "/bearing1_1_temp.json")

In [ ]:
"df -H" !

In [ ]:
dfData2.write.parquet("swift://coursera." + name + "/sample-data.parquet")

In [ ]:
"pwd" !

In [ ]:
"uname -a" !